# Creating Text-Fabric dataset (from GBI trees XML nodes)
The source data for the conversion are the XML node files representing the macula-greek version of Eberhard Nestle's 1904 Greek New Testament (British Foreign Bible Society, 1904). The starting dataset is formatted according to Syntax diagram markup by the Global Bible Initiative (GBI). The most recent source data can be found on github https://github.com/Clear-Bible/macula-greek/tree/main/Nestle1904/nodes. Attribution: "MACULA Greek Linguistic Datasets, available at https://github.com/Clear-Bible/macula-greek/". 

The production of the Text-Fabric files consist of two major steps. First the creation of pickle files (part 1). Secondly the actual Text-Fabric creation process (part 2). Both steps are independent, allowing to start from part 2 by using the pickle files as input. 

Please be advised that this Text-Fabric version is a test version (proof of concept) and may requires further finetuning, especialy with regards of nomenclature and presentation of (sub)phrases and clauses.

## Table of content <a class="anchor" id="TOC">
* [Part 1: Read GBI XML data and store in pickle](#first-bullet)
* [Part 2: Sort the nodes](#second-bullet)
* [Part 3: Nestle1904GBI production from pickle input](#third-bullet)

## Part 1: Read GBI XML data and store in pickle <a class="anchor" id="first-bullet"></a>
##### [Back to TOC](#TOC)

This script harvests all information from the GBI tree XML data file, puts it into a Panda DataFrame and stores the result per book in a pickle file. Note: pickling (in Python) is serialising an object into a disk file (or buffer).

In the context of this script, 'Leaf' refers to those node containing the Greek word as data, which happen to be the nodes without any child (hence the analogy with the leaves on the tree). These 'leafs' can also be refered to as 'terminal nodes'. Futher, Parent1 is the leaf's parent, Parent2 is Parent1's parent, etc.

For a full description of the source data see document [MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf](https://github.com/Clear-Bible/macula-greek/blob/main/doc/MACULA%20Greek%20Treebank%20for%20the%20Nestle%201904%20Greek%20New%20Testament.pdf)

### Step 1: import various libraries

In [1]:
import pandas as pd
import sys
import os
import time
import pickle

import re  #regular expressions
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

### Step 2: initialize global data

IMPORTANT: In case you want to build the Text-Fabric files on your own system, you need to change BaseDir, InputDir and OutputDir to match location of the datalocation and the OS used.

In [2]:
BaseDir = 'C:\\Users\\tonyj\\my_new_Jupyter_folder\\test_of_xml_etree\\'
InputDir = BaseDir+'inputfiles\\'
OutputDir = BaseDir+'outputfiles\\'

#         key: filename,       [0]=book_long,   [1]=book_num,  [3]=book_short
bo2book = {'01-matthew':       ['Matthew',         '1',        'Matt'],
           '02-mark':          ['Mark',            '2',        'Mark'],
           '03-luke':          ['Luke',            '3',        'Luke'],
           '04-john':          ['John',            '4',        'John'],
           '05-acts':          ['Acts',            '5',        'Acts'],
           '06-romans':        ['Romans',          '6',        'Rom'],
           '07-1corinthians':  ['I_Corinthians',   '7',        '1Cor'],
           '08-2corinthians':  ['II_Corinthians',  '8',        '2Cor'],
           '09-galatians':     ['Galatians',       '9',        'Gal'],
           '10-ephesians':     ['Ephesians',       '10',       'Eph'],
           '11-philippians':   ['Philippians',     '11',       'Phil'],
           '12-colossians':    ['Colossians',      '12',       'Col'],
           '13-1thessalonians':['I_Thessalonians', '13',       '1Thess'],
           '14-2thessalonians':['II_Thessalonians','14',       '2Thess'],
           '15-1timothy':      ['I_Timothy',       '15',       '1Tim'],
           '16-2timothy':      ['II_Timothy',      '16',       '2Tim'],
           '17-titus':         ['Titus',           '17',       'Titus'],
           '18-philemon':      ['Philemon',        '18',       'Phlm'],
           '19-hebrews':       ['Hebrews',         '19',       'Heb'],
           '20-james':         ['James',           '20',       'Jas'],
           '21-1peter':        ['I_Peter',         '21',       '1Pet'],
           '22-2peter':        ['II_Peter',        '22',       '2Pet'],
           '23-1john':         ['I_John',          '23',       '1John'],
           '24-2john':         ['II_John',         '24',       '2John'],
           '25-3john':         ['III_John',        '25',       '3John'],     
           '26-jude':          ['Jude',            '26',       'Jude'],
           '27-revelation':    ['Revelation',      '27',       'Rev']}

### Step 3: define Function to add parent info to each node of the XML tree

In order to traverse from the 'leafs' (terminating nodes) upto the root of the tree, it is required to add information to each node pointing to the parent of each node.

(concept taken from https://stackoverflow.com/questions/2170610/access-elementtree-node-parent-node)

In [3]:
def addParentInfo(et):
    for child in et:
        child.attrib['parent'] = et
        addParentInfo(child)

def getParent(et):
    if 'parent' in et.attrib:
        return et.attrib['parent']
    else:
        return None

### Step 4: read and process the XML data and store panda dataframe in pickle

In [ ]:
# set some globals
monad=1
CollectedItems= 0

# process books in order
for bo, bookinfo in bo2book.items():
  CollectedItems=0
  full_df=pd.DataFrame({})
  book_long=bookinfo[0]
  booknum=bookinfo[1]
  book_short=bookinfo[2]
  InputFile = os.path.join(InputDir, f'{bo}.xml')
  OutputFile = os.path.join(OutputDir, f'{bo}.pkl')
  print(f'Processing {book_long} at {InputFile}')

  # send xml document to parsing process
  tree = ET.parse(InputFile)
  # Now add all the parent info to the nodes in the xtree [important!]
  addParentInfo(tree.getroot())
  start_time = time.time()
    
  # walk over all the leaves and harvest the data
  for elem in tree.iter():
    if not list(elem):
        # if no child elements, this is a leaf/terminal node
     
        # show  progress on screen
        CollectedItems+=1
        if (CollectedItems%100==0): print (".",end='')
        
        #Leafref will contain list with book, chapter verse and wordnumber
        Leafref = re.sub(r'[!: ]'," ", elem.attrib.get('ref')).split()
        
        #push value for monad to element tree 
        elem.set('monad', monad)
        monad+=1
        
        # add some important computed data to the leaf
        elem.set('LeafName', elem.tag)
        elem.set('word', elem.text)
        elem.set('book_long', book_long)
        elem.set('booknum', int(booknum))
        elem.set('book_short', book_short)
        elem.set('chapter', int(Leafref[1]))
        elem.set('verse', int(Leafref[2]))
       
        # following code will trace down parents upto the tree and store found attributes
        parentnode=getParent(elem)
        index=0
        while (parentnode):
           index+=1
           elem.set('Parent{}Name'.format(index),  parentnode.tag)
           elem.set('Parent{}Type'.format(index),  parentnode.attrib.get('Type'))
           elem.set('Parent{}Cat'.format(index),   parentnode.attrib.get('Cat'))
           elem.set('Parent{}Start'.format(index), parentnode.attrib.get('Start'))
           elem.set('Parent{}End'.format(index),   parentnode.attrib.get('End'))
           elem.set('Parent{}Rule'.format(index),  parentnode.attrib.get('Rule'))
           elem.set('Parent{}Head'.format(index),  parentnode.attrib.get('Head'))
           elem.set('Parent{}NodeId'.format(index),parentnode.attrib.get('nodeId'))
           elem.set('Parent{}ClType'.format(index),parentnode.attrib.get('ClType'))
           elem.set('Parent{}HasDet'.format(index),parentnode.attrib.get('HasDet'))
           currentnode=parentnode
           parentnode=getParent(currentnode)      
        elem.set('parents', int(index))
        
        #this will push all elements found in the tree into a DataFrame
        df=pd.DataFrame(elem.attrib, index={monad})
        full_df=pd.concat([full_df,df])
        
  #store the resulting DataFrame per book into a pickle file for further processing
  df = df.convert_dtypes(convert_string=True)
  output = open(r"{}".format(OutputFile), 'wb')
  pickle.dump(full_df, output)
  output.close()
  print("\nFound ",CollectedItems, " items in  %s seconds\n" % (time.time() - start_time))  
        

## Part 2: Sort the nodes <a class="anchor" id="second-bullet"></a>
##### [Back to TOC](#TOC)


The node data is not the same as the word order in the running text. This part is to sort the dataframes accordingly.

In [15]:
BaseDir = 'C:\\Users\\tonyj\\my_new_Jupyter_folder\\test_of_xml_etree\\'
source_dir = BaseDir+'outputfiles\\'  #the input files (with 'wordjumps')
output_dir  = BaseDir+'outputfiles_sorted\\' #the output files (words in order of running text)


for bo in bo2book:
        '''
        load all data into a dataframe
        process books in order (bookinfo is a list!)
        '''   
        InputFile = os.path.join(source_dir, f'{bo}.pkl')
        OutputFile = os.path.join(output_dir, f'{bo}.pkl')
       
        print(f'\tloading {InputFile}...')
        pkl_file = open(InputFile, 'rb')
        df = pickle.load(pkl_file)
        pkl_file.close()
   
        # fill dictionary of column names for this book 
        IndexDict = {}             # init an empty dictionary
        ItemsInRow=1
        for itemname in df.columns.to_list():
            IndexDict.update({'i_{}'.format(itemname): ItemsInRow})
            ItemsInRow+=1
        
        # sort by id
        df.sort_values(by=['nodeId'])
        #store the resulting DataFrame per book into a pickle file for further processing
        #df = df.convert_dtypes(convert_string=True) DO NOT DO THIS! IT MUTILATES THE DATA...
        output = open(r"{}".format(OutputFile), 'wb')
        pickle.dump(df, output)
        output.close()
         
        

	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\01-matthew.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\02-mark.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\03-luke.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\04-john.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\05-acts.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\06-romans.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\07-1corinthians.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\08-2corinthians.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\09-galatians.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles\10-ephesians.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_e

## Part 3: Nestle1904GBI Text-Fabric production from pickle input <a class="anchor" id="second-bullet"></a>
##### [Back to TOC](#TOC)

This script creates the Text-Fabric files by recursive calling the TF walker function.
API info: https://annotation.github.io/text-fabric/tf/convert/walker.html

The pickle files created by step 1 are stored on Github location https://github.com/tonyjurg/Nestle1904GBI/tree/main/resources/picklefiles

### Step 1: Load libraries and initialize some data

Change BaseDir, InputDir and OutputDir to match location of the datalocation and the OS used.

In [2]:
import pandas as pd
import os
import re
import gc
from tf.fabric import Fabric
from tf.convert.walker import CV
from tf.parameters import VERSION
from datetime import date
import pickle


BaseDir = 'C:\\Users\\tonyj\\my_new_Jupyter_folder\\test_of_xml_etree\\'
source_dir = BaseDir+'outputfiles_sorted\\'  # the input for the walker is the output of the xml to excel 
#output_dir  = BaseDir+'outputfilesTF\\' #the TextFabric files
output_dir = 'C:\\text-fabric-data\\github\\tonyjurg\\Nestle1904GBI\\tf'

#         key: filename,       [0]=book_long,   [1]=book_num,  [3]=book_short
bo2book = {'01-matthew':       ['Matthew',         '1',        'Matt'],
           '02-mark':          ['Mark',            '2',        'Mark'],
           '03-luke':          ['Luke',            '3',        'Luke'],
           '04-john':          ['John',            '4',        'John'],
           '05-acts':          ['Acts',            '5',        'Acts'],
           '06-romans':        ['Romans',          '6',        'Rom'],
           '07-1corinthians':  ['I_Corinthians',   '7',        '1Cor'],
           '08-2corinthians':  ['II_Corinthians',  '8',        '2Cor'],
           '09-galatians':     ['Galatians',       '9',        'Gal'],
           '10-ephesians':     ['Ephesians',       '10',       'Eph'],
           '11-philippians':   ['Philippians',     '11',       'Phil'],
           '12-colossians':    ['Colossians',      '12',       'Col'],
           '13-1thessalonians':['I_Thessalonians', '13',       '1Thess'],
           '14-2thessalonians':['II_Thessalonians','14',       '2Thess'],
           '15-1timothy':      ['I_Timothy',       '15',       '1Tim'],
           '16-2timothy':      ['II_Timothy',      '16',       '2Tim'],
           '17-titus':         ['Titus',           '17',       'Titus'],
           '18-philemon':      ['Philemon',        '18',       'Phlm'],
           '19-hebrews':       ['Hebrews',         '19',       'Heb'],
           '20-james':         ['James',           '20',       'Jas'],
           '21-1peter':        ['I_Peter',         '21',       '1Pet'],
           '22-2peter':        ['II_Peter',        '22',       '2Pet'],
           '23-1john':         ['I_John',          '23',       '1John'],
           '24-2john':         ['II_John',         '24',       '2John'],
           '25-3john':         ['III_John',        '25',       '3John'],     
           '26-jude':          ['Jude',            '26',       'Jude'],
           '27-revelation':    ['Revelation',      '27',       'Rev']}


### Step 2 Running the Text-Fabric walker function

Text-Fabric API info can be found at https://annotation.github.io/text-fabric/tf/convert/walker.html

Explanatory notes regarding the logic of interpreting the data are included in the python code of the director function.

In [3]:
TF = Fabric(locations=output_dir, silent=False)
cv = CV(TF)
version = "0.2"

# the following is required to prevent passing float data to the walker function
def sanitize(input):
    if isinstance(input, float): return ''
    else: return (input)
   

def director(cv):
    
    NoneType = type(None)      # needed as tool to validate certain data
    prev_book = "Matthew"      # start at first book
    IndexDict = {}             # init an empty dictionary

    for bo,bookinfo in bo2book.items():
        
        # load all data into a dataframe and process books in order (bookinfo is a list!)
        book=bookinfo[0]  
        booknum=int(bookinfo[1])
        book_short=bookinfo[2]
        book_loc = os.path.join(source_dir, f'{bo}.pkl') 
        
        # Report progress or reading data
        print(f'\tloading {book_loc}...')
        pkl_file = open(book_loc, 'rb')
        df = pickle.load(pkl_file)
        pkl_file.close()
      
        # reset/load the following initial  variables (we are at the start of a new book)     
        phrasefunction = prev_phrasefunction = phrasefunction_long = prev_phrasefunction_long='TBD'
        this_clausetype = this_clauserule = phrasetype="unknown"
        prev_chapter = prev_verse = prev_sentence = prev_clause = prev_phrase = int(1)      
        sentence_track = clause_track = phrase_track = 1
        sentence_done = clause_done = phrase_done = verse_done = chapter_done = book_done = False  
        wrdnum = 0 # start at 0

        # build a dictionary of column names for this book
        ItemsInRow=1
        for itemname in df.columns.to_list():
            IndexDict.update({'i_{}'.format(itemname): ItemsInRow})
            ItemsInRow+=1
         
        
        # Create a set of nodes at the start a new book
        book_done = chapter_done = verse_done = phrase_done = clause_done = sentence_done = False
        this_book = cv.node('book')
        cv.feature(this_book, book=book, booknum=booknum, book_short=book_short)
        this_chapter = cv.node('chapter')
        cv.feature(this_chapter, chapter=1)
        this_verse = cv.node('verse')
        cv.feature(this_verse, verse=1)
        this_sentence = cv.node('sentence')
        cv.feature(this_sentence, sentence=1)
        this_clause = cv.node('clause')
        this_phrase = cv.node('phrase')
        
        
        '''
        Walks through the texts and triggers
        slot and node creation events.
        '''
        
        # iterate through words and construct objects
        for row in df.itertuples():
            wrdnum += 1
           
            # get number of parent nodes for this word
            parents = row[IndexDict.get("i_parents")]
      
            # get chapter and verse for this word from the data
            chapter = row[IndexDict.get("i_chapter")]
            verse = row[IndexDict.get("i_verse")]
            
            # get clause rule and type info of parent clause
            for i in range(1,parents-1):
                 item = IndexDict.get("i_Parent{}Cat".format(i))
                 if row[item]=="CL":
                    clauseparent=i
                    this_clauserule=row[IndexDict.get("i_Parent{}Rule".format(i))]  
                    this_clausetype=row[IndexDict.get("i_Parent{}ClType".format(i))]  
                    break
            cv.feature(this_clause, clause=clause_track, clauserule=this_clauserule, clausetype=this_clausetype)
            

            # get phrase type info
            prev_phrasetype=phrasetype
            for i in range(1,parents-1):
                 item = IndexDict.get("i_Parent{}Cat".format(i))
                 if row[item]=="np":
                    _item ="i_Parent{}Rule".format(i)
                    phrasetype=row[IndexDict.get(_item)]
                    break
            functionaltag=row[IndexDict.get('i_FunctionalTag')]

            
            # determine syntactic categories of clause parts. See also the description in 
            # "MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf" page 5&6
            # (section 2.4 Syntactic Categories at Clause Level)
            phrase_done = False
            for i in range(1,clauseparent):              
                phrasefunction = row[IndexDict.get("i_Parent{}Cat".format(i))]                  
                if phrasefunction=="ADV":
                    phrasefunction_long='Adverbial function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="IO":
                    phrasefunction_long='Indirect Object function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="O":
                    phrasefunction_long='Object function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="O2":
                    phrasefunction_long='Second Object function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="S":
                    phrasefunction_long='Subject function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=='P':
                    phrasefunction_long='Predicate function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="V":
                    phrasefunction_long='Verbal function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break
                elif phrasefunction=="VC":
                    phrasefunction_long='Verbal Copula function'
                    if prev_phrasefunction!=phrasefunction: phrase_done = True
                    break


            # determine syntactic categories at word level. See also the description in 
            # "MACULA Greek Treebank for the Nestle 1904 Greek New Testament.pdf" page 6&7
            # (2.2. Syntactic Categories at Word Level: Part of Speech Labels)
            sp=sanitize(row[IndexDict.get("i_Cat")])
            if sp=='adj':
                sp_full='adjective'
            elif sp=='adj':
                sp_full='adjective'
            elif sp=='conj':
                sp_full='conjunction'
            elif sp=='det':
                sp_full='determiner' 
            elif sp=='intj':
                sp_full='interjection' 
            elif sp=='noun':
                sp_full='noun' 
            elif sp=='num':
                sp_full='numeral' 
            elif sp=='prep':
                sp_full='preposition' 
            elif sp=='ptcl':
                sp_full='particle' 
            elif sp=='pron':
                sp_full='pronoun' 
            elif sp=='verb':
                sp_full='verb' 
                    
                    
            '''
            determine if conditions are met to trigger some action 
            action will be executed after next word
            '''   
 
            # detect chapter boundary
            if prev_chapter != chapter:
                chapter_done = True
                verse_done=True
                sentence_done = True
                clause_done = True
                phrase_done = True
                
            # detect verse boundary
            if prev_verse != verse:
                verse_done=True
                
                       
            
            '''
             -- handle TF events --
            Determine what actions need to be done if proper condition is met.
            '''   

            # act upon end of phrase (close)
            if phrase_done or clause_done:
                cv.feature(this_phrase, phrase=phrase_track, phrasetype=prev_phrasetype, phrasefunction=prev_phrasefunction, phrasefunction_long=prev_phrasefunction_long)
                cv.terminate(this_phrase)
                prev_phrasefunction=phrasefunction
                prev_phrasefunction_long=phrasefunction_long
                
            # act upon end of clause (close)  
            if clause_done:
                cv.terminate(this_clause)
 
            # act upon end of sentence (close)
            if sentence_done:
                cv.terminate(this_sentence)
                
            # act upon end of verse (close)
            if verse_done:

                cv.terminate(this_verse)
                prev_verse = verse  

            # act upon end of chapter (close)
            if chapter_done:

                cv.terminate(this_chapter)
                prev_chapter = chapter

                
            # start of chapter (create new)
            if chapter_done:
                this_chapter = cv.node('chapter')
                cv.feature(this_chapter, chapter=chapter)
                chapter_done = False
                
            # start of verse (create new)
            if verse_done:
                this_verse = cv.node('verse')
                cv.feature(this_verse, verse=verse)
                verse_done = False  
                
            # start of sentence (create new)
            if sentence_done:
                this_sentence= cv.node('sentence')
                cv.feature(this_sentence, sentence=sentence_track)
                sentence_track += 1
                sentence_done = False

                
            # start of clause (create new)  
            if clause_done:
                this_clause = cv.node('clause')
                cv.feature(this_clause, clause=clause_track, clauserule=this_clauserule,clausetype=this_clausetype)
                clause_track += 1
                clause_done = False
                phrase_done = True 

                
            # start of phrase (create new)
            if phrase_done:
                this_phrase = cv.node('phrase')
                prev_phrase = phrase_track
                prev_phrasefunction=phrasefunction
                prev_phrasefunction_long=phrasefunction_long
                phrase_track += 1
                phrase_done = False
                
      
            # Detect boundaries of sentences, clauses and phrases 
            text=row[IndexDict.get("i_Unicode")]
            if text[-1:] == "." :        
                sentence_done = True
                clause_done = True
                phrase_done = True
            if text[-1:] == ";" or text[-1:] == ",":
                clause_done = True
                phrase_done = True       
                    
            
            '''
            -- create word nodes --
            '''   
                    
            # some attributes are not present inside some (small) books. The following is to prevent exceptions.
            degree='' 
            if 'i_Degree' in IndexDict: 
                   degree=sanitize(row[IndexDict.get("i_Degree")]) 
            subjref=''
            if 'i_SubjRef' in IndexDict:
                   subjref=sanitize(row[IndexDict.get("i_SubjRef")]) 
        

            # make word object
            this_word = cv.slot()
            cv.feature(this_word, 
                    word=row[IndexDict.get("i_Unicode")],
                    monad=row[IndexDict.get("i_monad")],
                    orig_order=row[IndexDict.get("i_monad")],
                    book_long=row[IndexDict.get("i_book_long")],
                    booknum=booknum,
                    book_short=row[IndexDict.get("i_book_short")],
                    chapter=chapter,
                    sp=sp,
                    sp_full=sp_full,
                    verse=verse,
                    sentence=sentence_track,
                    clause=clause_track,
                    phrase=phrase_track,
                    normalized=sanitize(row[IndexDict.get("i_NormalizedForm")]),
                    formaltag=sanitize(row[IndexDict.get("i_FormalTag")]),
                    functionaltag=functionaltag,
                    strongs=sanitize(row[IndexDict.get("i_StrongNumber")]),
                    lex_dom=sanitize(row[IndexDict.get("i_LexDomain")]),
                    ln=sanitize(row[IndexDict.get("i_LN")]),
                    gloss_EN=sanitize(row[IndexDict.get("i_Gloss")]),
                    gn=sanitize(row[IndexDict.get("i_Gender")]),
                    nu=sanitize(row[IndexDict.get("i_Number")]),
                    case=sanitize(row[IndexDict.get("i_Case")]),
                    lemma=sanitize(row[IndexDict.get("i_UnicodeLemma")]),
                    person=sanitize(row[IndexDict.get("i_Person")]),
                    mood=sanitize(row[IndexDict.get("i_Mood")]),
                    tense=sanitize(row[IndexDict.get("i_Tense")]),
                    number=sanitize(row[IndexDict.get("i_Number")]),
                    voice=sanitize(row[IndexDict.get("i_Voice")]),
                    degree=degree,
                    type=sanitize(row[IndexDict.get("i_Type")]),
                    reference=sanitize(row[IndexDict.get("i_Ref")]),     # the capital R is critical here!
                    subj_ref=subjref,
                    nodeID=row[1]                              #this is a fixed position.
                    )
            cv.terminate(this_word)

        
        '''
        -- wrap up the book --
        '''   
        
        # close all nodes (phrase, clause, sentence, verse, chapter and book)
        cv.feature(this_phrase, phrase=phrase_track, phrasetype=prev_phrasetype,phrasefunction=prev_phrasefunction,phrasefunction_long=prev_phrasefunction_long)
        cv.terminate(this_phrase)
        cv.feature(this_clause, clause=clause_track, clauserule=this_clauserule, clausetype=this_clausetype)
        cv.terminate(this_clause)
        cv.feature(this_sentence, sentence=prev_sentence)
        cv.terminate(this_sentence)
        cv.feature(this_verse, verse=prev_verse)
        cv.terminate(this_verse)
        cv.feature(this_chapter, chapter=prev_chapter)
        cv.terminate(this_chapter)
        cv.feature(this_book, book=prev_book)
        cv.terminate(this_book)
        
        # clear dataframe for this book 
        del df
        # clear the index dictionary
        IndexDict.clear()
        gc.collect()
        
        
'''
-- output definitions --
'''  
        
slotType = 'word'  # or whatever you choose
otext = {  # dictionary of config data for sections and text formats
        'fmt:text-orig-full':'{word}',
        'sectionTypes':'book,chapter,verse',
        'sectionFeatures':'book,chapter,verse',
        'structureFeatures': 'book,chapter,verse',
        'structureTypes': 'book,chapter,verse',
        }

# configure metadata
generic = {  # dictionary of metadata meant for all features
         'Name': 'Greek New Testament (Nestle 1904) based upon GBI tree node data',
         'Version': '{}'.format(version),
         'Editors': 'Eberhart Nestle',
         'Data source': 'MACULA Greek Linguistic Datasets, available at https://github.com/Clear-Bible/macula-greek/tree/main/Nestle1904/nodes',
         'Availability': 'Creative Commons Attribution 4.0 International (CC BY 4.0)', 
         'Converter_author': 'Tony Jurg, ReMa student Vrije Universiteit Amsterdam, Netherlands', 
         'Converter_execution': 'Tony Jurg, ReMa student Vrije Universiteit Amsterdam, Netherlands', 
         'Convertor_source': 'https://github.com/tonyjurg/Nestle1904GBI/tree/main/resources/converter',
         'TextFabric version': '{}'.format(VERSION)  #imported from tf.parameters
         }

intFeatures = {  # set of integer valued feature names
         'booknum',
         'chapter',
         'verse',
         'sentence',
         'clause',
         'phrase',
         'orig_order',
         'monad'
         }

featureMeta = {  # per feature dicts with metadata
                 'book': {'description': 'Book'},
                 'book_long': {'description': 'Book name (fully spelled out)'},
                 'booknum': {'description': 'NT book number (Matthew=1, Mark=2, ..., Revelation=27)'},
                 'book_short': {'description': 'Book name (abbreviated)'},
                 'chapter': {'description': 'Chapter number inside book'},
                 'verse': {'description': 'Verse number inside chapter'},
                 'sentence': {'description': 'Sentence number (counted per chapter)'},
                 'clause': {'description': 'Clause number (counted per chapter)'},
                 'clauserule': {'description': 'Clause rule'},
                 'clausetype': {'description': 'Clause type'},
                 'phrase' : {'description': 'Phrase number (counted per chapter)'},
                 'phrasetype' : {'description': 'Phrase type information'},
                 'phrasefunction' : {'description': 'Phrase function (abbreviated)'},
                 'phrasefunction_long' : {'description': 'Phrase function (long description)'},
                 'orig_order': {'description': 'Word order within corpus'},
                 'monad':{'description': 'Monad'},
                 'word': {'description': 'Word as it appears in the text'},
                 'sp': {'description': 'Part of Speech (abbreviated)'},
                 'sp_full': {'description': 'Part of Speech (long description)'}, 
                 'normalized': {'description': 'Surface word stripped of punctations'},
                 'lemma': {'description': 'Lexeme (lemma)'},
                 'formaltag': {'description': 'Formal tag (Sandborg-Petersen morphology)'},
                 'functionaltag': {'description': 'Functional tag (Sandborg-Petersen morphology)'},
                 # see also discussion on relation between lex_dom and ln @ https://github.com/Clear-Bible/macula-greek/issues/29
                 'lex_dom': {'description': 'Lexical domain according to Semantic Dictionary of Biblical Greek, SDBG (not present everywhere?)'},
                 'ln': {'description': 'Lauw-Nida lexical classification (not present everywhere?)'},
                 'strongs': {'description': 'Strongs number'},
                 'gloss_EN': {'description': 'English gloss'},
                 'gn': {'description': 'Gramatical gender (Masculine, Feminine, Neuter)'},
                 'nu': {'description': 'Gramatical number (Singular, Plural)'},
                 'case': {'description': 'Gramatical case (Nominative, Genitive, Dative, Accusative, Vocative)'},
                 'person': {'description': 'Gramatical person of the verb (first, second, third)'},
                 'mood': {'description': 'Gramatical mood of the verb (passive, etc)'},
                 'tense': {'description': 'Gramatical tense of the verb (e.g. Present, Aorist)'},
                 'number': {'description': 'Gramatical number of the verb'},
                 'voice': {'description': 'Gramatical voice of the verb'},
                 'degree': {'description': 'Degree (e.g. Comparitative, Superlative)'},
                 'type': {'description': 'Gramatical type  of noun or pronoun (e.g. Common, Personal)'},
                 'reference': {'description': 'Reference (to nodeID in XML source data, not yet post-processes)'},
                 'subj_ref': {'description': 'Subject reference (to nodeID in XML source data, not yet post-processes)'},
                 'nodeID': {'description': 'Node ID (as in the XML source data, not yet post-processes)'}
                 }

'''
 -- the main function  --
'''   


good = cv.walk(
    director,
    slotType,
    otext=otext,
    generic=generic,
    intFeatures=intFeatures,
    featureMeta=featureMeta,
    warn=True,
    force=False
)

if good:
  print ("done")

This is Text-Fabric 11.4.10
0 features found and 0 ignored
  0.00s Not all of the warp features otype and oslots are present in
C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
  0.00s Only the Feature and Edge APIs will be enabled
  0.00s Warp feature "otext" not found. Working without Text-API

  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |   SECTION   TYPES:    book, chapter, verse
   |   SECTION   FEATURES: book, chapter, verse
   |   STRUCTURE TYPES:    book, chapter, verse
   |   STRUCTURE FEATURES: book, chapter, verse
   |   TEXT      FEATURES:
   |      |   text-orig-full       word
   |     0.00s OK
   |     0.00s Following director... 
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles_sorted\01-matthew.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles_sorted\02-mark.pkl...
	loading C:\Users\tonyj\my_new_Jupyter_folder\test_of_xml_etree\outputfiles_sorted\03-l

   |     0.02s T clauserule           to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.01s T clausetype           to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.15s T degree               to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.14s T formaltag            to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.13s T functionaltag        to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.14s T gloss_EN             to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.14s T gn                   to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.15s T lemma                to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.13s T lex_dom              to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.14s T ln                   to C:/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf
   |     0.13s T monad                to C:/text-fabric-data